In [1]:
!nvidia-smi

Tue Aug 15 23:14:46 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.66                 Driver Version: 375.66                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:04:00.0     Off |                  N/A |
| 27%   29C    P8     9W / 180W |   1355MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 0000:05:00.0     Off |                  N/A |
| 36%   53C    P2    51W / 180W |   7163MiB /  8114MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
%matplotlib notebook
import keras
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.layers import TimeDistributed, Dense, LSTM, Activation
from keras.layers import Dropout, GRU, Bidirectional, Conv2D, Conv1D
from keras.layers import MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2, l1
from keras.models import Sequential
from tracker3d import loader, utils, metrics

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:0E:00.0)


In [3]:
%%time
order        = ("r", "phi", "z")
code         = (order[0][0] + order[1][0] + order[2][0]).upper()
n_noise      = 0
num_classes  = (25 + 2)
input_shape  = ((25 * 9), 3)
load_file    = np.load("datasets/npz/UNIF-25T-1000E-{0}-{1}N.npz".format(code, n_noise))
train_data   = pad_sequences(load_file["arr_0"], maxlen=input_shape[0], padding="post", dtype=float)
train_target = [utils.to_categorical(seq, num_classes) for seq in load_file["arr_1"]]
train_target = pad_sequences(train_target, maxlen=(9 * 25), padding="post")
opt          = "rmsprop"

for matrix in train_target:
    for row in matrix:
        if row.sum() == 0:
            row[-1] = 1

In [ ]:
utils.display_side_by_side(train_data[3], train_target[3], order=order)

In [5]:
%%time
model = Sequential()
model.add(Bidirectional(GRU(256, return_sequences=True, recurrent_dropout=0.4, implementation=2, activity_regularizer=l1(0.01)), input_shape=input_shape, merge_mode="mul"))
model.add(Dropout(0.4))
model.add(Bidirectional(GRU(256, return_sequences=True, recurrent_dropout=0.4, implementation=2, activity_regularizer=l1(0.01)), merge_mode="mul"))
model.add(Dropout(0.4))
model.add(TimeDistributed(Dense(num_classes, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 225, 256)          399360    
_________________________________________________________________
dropout_3 (Dropout)          (None, 225, 256)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 225, 256)          787968    
_________________________________________________________________
dropout_4 (Dropout)          (None, 225, 256)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 225, 27)           6939      
Total params: 1,194,267
Trainable params: 1,194,267
Non-trainable params: 0
_________________________________________________________________
CPU times: user 4.2 s, sys: 15.5 s, total: 19.7 s
Wall time: 2.54 s


In [7]:
%%time
epochs = 50
batch_size = 100
valsplit = 0.1
modelpath = "models/BIG-1000E-SLIDES-{0}-{1}N.h5".format(code, n_noise)
hist = model.fit (
    train_data,
    train_target,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    validation_split=valsplit,
    callbacks=[keras.callbacks.ModelCheckpoint(
        filepath=modelpath,
        save_best_only=True
    )]
)

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Train on 22500 samples, validate on 2500 samples
Epoch 1/50
22500/22500 [==============================] - 137s - loss: 788.5050 - acc: 0.5133 - val_loss: 27.2606 - val_acc: 0.5085
Epoch 2/50
22500/22500 [==============================] - 137s - loss: 20.5923 - acc: 0.5154 - val_loss: 16.3079 - val_acc: 0.5085
Epoch 3/50
22500/22500 [==============================] - 136s - loss: 13.0212 - acc: 0.5154 - val_loss: 20.8069 - val_acc: 0.5085
Epoch 4/50
22500/22500 [==============================] - 136s - loss: 10.3397 - acc: 0.5154 - val_loss: 5.5149 - val_acc: 0.5085
Epoch 5/50
22500/22500 [==============================] - 136s - loss: 6.7829 - acc: 0.5154 - val_loss: 7.0614 - val_acc: 0.5085
Epoch 6/50
22500/22500 [==============================] - 135s - loss: 6.4272 - acc: 0.5154 - val_loss: 9.0126 - val_acc: 0.5085
Epoch 7/50
22500/22500 [==============================] - 135s - loss: 6.3117 - acc: 0.5154 - val_loss: 2.5953 - val_acc: 0.5085
Epoch 8/50
22500/22500 [================